In [65]:
out_dir = r'/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/collaborations/Aphantasia_Kutsche/resources'

In [ ]:
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'

In [1]:
#Enter coordinates of sphere
x = -42
y = -54
z = 18
radius=5

# Option 1 - Custom

Generate the Sphere at The Coordinate

In [37]:
import os
import numpy as np
import nibabel as nib
from nilearn import image, plotting

def convert_ijk_to_mni(ijk, affine):
    return affine @ np.append(ijk, 1)

def convert_mni_to_ijk(mni, affine):
    inv_affine = np.linalg.pinv(affine)
    return inv_affine @ np.append(mni, 1)

def calculate_voxelwise_mni_coords(brain_img):
    brain_mask = brain_img.get_fdata()
    ijk_mask = np.array(np.meshgrid(
        np.arange(brain_mask.shape[0]),
        np.arange(brain_mask.shape[1]),
        np.arange(brain_mask.shape[2]),
        indexing='ij'
    ))                                                                                  #shape (3, i, j, k). An array stacking the ijk coords into 3 tensors
    ijk_arr = ijk_mask.reshape(3,-1).T                                                  #shape (n, 3). An array with the ijk coords in sub-arrays. 
    ijk1 = np.zeros((ijk_arr.shape[0],4))                                               #shape (n,4)
    ijk1[:,:3] = ijk_arr                                                                #shape (n,4). An array with the ijk coords in sub-arrays
    ijk1[:, 3] = 1                                                                      #shape (n,4). Prepared for homologous transform
    affine = brain_img.affine                                                           #shape (4,4). The affine matrix of the brain mask
    mni_coords = np.dot(affine, ijk1.T).T[:, :3]                                        #shape (n,3). The MNI coordinates of each voxel
    return mni_coords

def calculate_pairwise_distance(mni_coords_point, mni_coords_brain):
    dist_from_point = np.linalg.norm(mni_coords_brain - mni_coords_point, axis=1)       #shape (n,). The euclidean distance from the point to each voxel
    return dist_from_point

def points_within_sphere(distances, radius):
    return distances <= radius

def generate_sphere_mask(points, brain_img):
    return np.reshape(points, brain_img.get_fdata().shape)

def save_mask(mask, brain_img, mni_arr, out_path):
    affine = brain_img.affine
    mask_img = nib.Nifti1Image(mask, affine)
    html_img = plotting.view_img(mask_img, cut_coords=(mni_arr[0],mni_arr[1],mni_arr[2]), black_bg=False, opacity=.75, cmap='ocean_hot')
    mask_img.to_filename(os.path.join(out_path, 'sphere_roi'))


In [ ]:
# Load the brain mask image
brain_img = nib.load(mask_path)
mni_coords_brain = calculate_voxelwise_mni_coords(brain_img)
distances = calculate_pairwise_distance(np.array[x,y,z], mni_coords_brain)
points_in_sphere = points_within_sphere(distances, radius)
sphere_mask = generate_sphere_mask(points_in_sphere, brain_img)

# Save the generated sphere mask
save_mask(sphere_mask, brain_img, np.array[x,y,z], out_path=out_dir)

Use NLTools

In [1]:
from nltools.mask import create_sphere
def gen_sphere_roi(xcoord, ycoord, zcoord, mask=None, radius=4):
    sphere_mask = create_sphere([xcoord, ycoord, zcoord], radius=radius, mask=mask)
    return sphere_mask.get_fdata()


ModuleNotFoundError: No module named 'nltools'

In [ ]:
#Visualize
import nibabel as nib
from nilearn import image, plotting, maskers
mask = nib.load('/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_all_ones.nii.gz')

sphere_roi = gen_sphere_roi(xcoord=x, ycoord=y, zcoord=z, mask=mask, radius=radius);
ovr_img3 = image.new_img_like(mask, sphere_roi);
ovr_html3 = plotting.view_img(ovr_img3, cut_coords=(0,0,0), black_bg=False, opacity=.75, cmap='ocean_hot');
ovr_html3

In [ ]:
#Save
import os
if os.path.isdir(out_dir)==False:
    os.mkdir(out_dir)

#Save
ovr_img3.to_filename(os.path.join(out_dir, f'{analysis}'))
ovr_html3.save_as_html(os.path.join(out_dir, f'{analysis}.html'))
print('saved to: ', os.path.join(out_dir, f'{analysis}'))
